<a href="https://colab.research.google.com/github/pacayton/Stat_268_Advanced_Time_Series_Analysis/blob/main/Stat_268_Week2_Intervention_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Stat 268: Advanced Time Series Analysis
$$\quad$$
## Assoc Prof Peter Julian Cayton, PhD
## School of Statistics
## University of the Philippines Diliman
## [Google Scholar](https://scholar.google.com/citations?user=KtH_mGEAAAAJ)




# Interrupted Time Series Analysis

*   Also known as "intervention analysis".
*   It is a system of methods in time series analysis to extract insights on the nature and impact of interventions in the behavior of the phenomenon as observed from the time series data.
* It is also a field in causal inference in which the causal effect of interventions, such as policies, are analyzed from the shifts in the behavior observed from time series data.

## Interventions

* events in the history of the phenomenon that leads to changes in behavior that are observable in the time series data.
* These changes may be abrupt or gradual, and their impact may be permanent or temporary
* Examples: policy changes, new laws, changes in government/management, reforms, change in practices, mergers and acquisitions, weather events.

